In [ ]:
import csv
import gc
import cv2
from tensorflow import keras
import numpy as np
from keras.models import load_model
import matplotlib.pyplot as plt
import neptune
from tensorflow.python.client import device_lib
from PIL import Image
import tensorflow as tf
import os
from keras.layers import LeakyReLU
from decouple import config
import matplotlib
from matplotlib import cm
import itertools
from pathlib import Path
from scipy.io import savemat

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
print(device_lib.list_local_devices())

In [ ]:
########################################################################################################################
# Ijjeh Model
########################################################################################################################
os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/h5_models/')
new_dim = 32
CR = (new_dim ** 2) / (69 ** 2)  # Compression Ratio
rescale_factor = int(512 / new_dim)

# model_name = 'super_resolution_AE_rescale_factor_%d_Uniform_grid_32_32_input.h5' % rescale_fact$or
model_name = 'super_resolution_AE_rescale_factor_%d_Uniform_grid_32_32_adding_GRL.h5' % rescale_factor  # Used in DLSS paper
# model_name = 'super_resolution_AE_rescale_factor_%d_Uniform_grid_32_32_adding_GRL_custom_loss_FFT2D.h5'% rescale_factor
model = load_model(model_name, compile=False)
model.summary()

########################################################################################################################
# Saeed Model
########################################################################################################################
# os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Saeed_model/h5_model')
# new_dim = 32
# CR = (new_dim ** 2) / (69 ** 2)  # Compression Ratio
# rescale_factor = int(512 / new_dim)
# model_name = 'Sub_pix_latest.h5'
# model = load_model(model_name, compile=False)
# model.summary()
########################################################################################################################

In [ ]:
########################################################################################################################
# Load dataset
########################################################################################################################

def load_dataset(x):    
    os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Numerical/Numerical_test_set/')            
    Full_W_dataset = np.load('GT_Num_output_case_%d.npy' % x)
    CS_arr = np.load('Sample_Num_output_case_%d.npy' % x)    
    return CS_arr, Full_W_dataset  

In [ ]:
def testing():   
    for test_case in [438, 448, 456, 475]:
        x_test, y_test = load_dataset(test_case)                        
        prediction = model.predict(x_test, batch_size=1)
        prediction = np.asarray(prediction)
        frames = x_test.shape[0]        
        print(test_case)
        for i in range(frames):
            SR_pred = prediction[i].astype('float32')            
            original = x_test[i].astype('float32')
            GT_label_input = y_test[i].astype('float32')
            ############################################################################################################
            # plt.figure(figsize=(1, 1), dpi=512)
            # plt.gca().set_axis_off()
            # plt.axis('off')
            # plt.subplots_adjust(left=0.0, bottom=0.0, right=1, top=1.0, wspace=0.0, hspace=0.0)
            # plt.margins(0, 0)
            # plt.gca().xaxis.set_major_locator(plt.NullLocator())
            # plt.gca().yaxis.set_major_locator(plt.NullLocator())
            ############################################################################################################          
            # SR_pred = SR_pred * 255
            # SR_pred= SR_pred.astype(np.uint8)
            # plt.imshow(SR_pred)        
            # plt.axis('off')
            # plt.close('all')
            path = '/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Numerical/Num_mat_files/%d_output/' % test_case
            # path = '/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Numerical/Numerical_results_Full_wavefield/%d_output/' % test_case
            Path(path).mkdir(parents=True, exist_ok=True)
            os.chdir(path)  
            # SR_pred_mat = np.asmatrix(SR_pred)
            temp = {'f': SR_pred}
            savemat('SR_num_pred_output_%d_frame_%d.mat' % (i+1, test_case), temp)
            # print(np.max(SR_pred_mat))
            # cv2.imwrite('SR_Pred_output_%d_frame_output_%d.png' % (i+1, test_case), SR_pred)
            # cv2.imwrite('Ijjeh_SR_Pred_output_%d_frame_output_%d.png' % (i+1, test_case), SR_pred)
           

In [ ]:
if __name__ == '__main__':
    testing()

In [ ]:
import scipy.io
os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Numerical/Num_mat_files/438_output/')

mat = scipy.io.loadmat('SR_num_pred_output_104_frame_438.mat')

In [ ]:
print(mat)